# Change of Chararacter CHOCH

### Load the data

In [58]:
import pandas as pd
import pandas_ta as ta
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy import stats

# Enable plotly rendering 
import plotly.io as pio
pio.renderers.default = 'iframe' # or 'colab' or 'iframe' or 'iframe_connected' or 'sphinx_gallery'

#df = pd.read_csv("EURUSD_Candlestick_1_Hour_BID_04.05.2003-15.04.2023.csv")
df = pd.read_csv("tesla.candlestick_1_day.05072019.02092023.csv")
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)

df['RSI'] = ta.rsi(df.close, length=12)
df['EMA'] = ta.ema(df.close, length=150)
df.tail()

#df=df[0:5000]

,Date,open,high,low,close,volume,Dividends,Stock Splits,RSI,EMA
1195,2024-02-05 00:00:00-05:00,184.259995,184.679993,175.009995,181.059998,134294400,0.0,0.0,23.032076,229.170076
1196,2024-02-06 00:00:00-05:00,177.210007,186.490005,177.110001,185.100006,122676000,0.0,0.0,29.055034,228.586367
1197,2024-02-07 00:00:00-05:00,188.179993,189.789993,182.679993,187.580002,111535200,0.0,0.0,32.587653,228.043236
1198,2024-02-08 00:00:00-05:00,189.000000,191.619995,185.580002,189.559998,82927600,0.0,0.0,35.389721,227.533524
1199,2024-02-09 00:00:00-05:00,190.179993,194.119904,189.490005,191.561600,50361904,0.0,0.0,38.221620,227.057075


### Trend detection

In [59]:
EMAsignal = [0]*len(df)
backcandles = 15

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if max(df.open[i], df.close[i])>=df.EMA[i]:
            dnt=0
        if min(df.open[i], df.close[i])<=df.EMA[i]:
            upt=0
    if upt==1 and dnt==1:
        EMAsignal[row]=3
    elif upt==1:
        EMAsignal[row]=2
    elif dnt==1:
        EMAsignal[row]=1

df['EMASignal'] = EMAsignal

In [60]:
def isPivot(candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot high, 2 if pivot low, 3 if both and 0 default
    """
    if candle-window < 0 or candle+window >= len(df):
        return 0
    
    pivotHigh = 1
    pivotLow = 2
    for i in range(candle-window, candle+window+1):
        if df.iloc[candle].low > df.iloc[i].low:
            pivotLow=0
        if df.iloc[candle].high < df.iloc[i].high:
            pivotHigh=0
    if (pivotHigh and pivotLow):
        return 3
    elif pivotHigh:
        return pivotHigh
    elif pivotLow:
        return pivotLow
    else:
        return 0

In [61]:
window=5
df['isPivot'] = df.apply(lambda x: isPivot(x.name,window), axis=1)

In [62]:
def pointpos(x):
    if x['isPivot']==2:
        return x['low']-1e-3
    elif x['isPivot']==1:
        return x['high']+1e-3
    else:
        return np.nan
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [67]:
dfpl = df[700:800]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [64]:
def detect_structure(candle, backcandles, window):
    """
    Attention! window should always be greater than the pivot window! to avoid look ahead bias
    """
    localdf = df[candle-backcandles-window:candle-window]  
    highs = localdf[localdf['isPivot'] == 1].high.tail(3).values
    idxhighs = localdf[localdf['isPivot'] == 1].high.tail(3).index
    lows = localdf[localdf['isPivot'] == 2].low.tail(3).values
    idxlows = localdf[localdf['isPivot'] == 2].low.tail(3).index

    pattern_detected = False

    lim1 = 0.005
    lim2 = lim1/3
    if len(highs) == 3 and len(lows) == 3:
        order_condition = (idxlows[0] < idxhighs[0] 
                           < idxlows[1] < idxhighs[1] 
                           < idxlows[2] < idxhighs[2])
        diff_condition = ( 
                            abs(lows[0]-highs[0])>lim1 and 
                            abs(highs[0]-lows[1])>lim2 and
                            abs(highs[1]-lows[1])>lim1 and
                            abs(highs[1]-lows[2])>lim2
                            )
        pattern_1 = (lows[0] < highs[0] and
            lows[1] > lows[0] and lows[1] < highs[0] and
            highs[1] > highs[0] and
            lows[2] > lows[1] and lows[2] < highs[1] and
            highs[2] < highs[1] and highs[2] > lows[2]
            )

        pattern_2 = (lows[0] < highs[0] and
            lows[1] > lows[0] and lows[1] < highs[0] and
            highs[1] > highs[0] and
            lows[2] < lows[1] and
            highs[2] < highs[1] 
            )

        if (order_condition and
            diff_condition and
            (pattern_1 or pattern_2)
        ):
            pattern_detected = True

    if pattern_detected:
        return 1
    else:
        return 0

In [65]:
df['pattern_detected'] = df.index.map(lambda x: detect_structure(x, backcandles=40, window=6))

In [66]:
df[df['pattern_detected']!=0]

,Date,open,high,low,close,volume,Dividends,Stock Splits,RSI,EMA,EMASignal,isPivot,pointpos,pattern_detected
263,2020-05-21 00:00:00-04:00,54.400002,55.500000,53.066666,55.173332,183819000,0.0,0.0,63.252483,37.967369,2,0,NaN,1
264,2020-05-22 00:00:00-04:00,54.811333,55.452000,54.133331,54.458668,149812500,0.0,0.0,60.290204,38.185797,2,0,NaN,1
265,2020-05-26 00:00:00-04:00,55.633331,55.639999,54.380669,54.591331,121345500,0.0,0.0,60.663270,38.403089,2,0,NaN,1
266,2020-05-27 00:00:00-04:00,54.723999,55.180668,52.333332,54.681999,173242500,0.0,0.0,60.936887,38.618704,2,2,52.332332,1
267,2020-05-28 00:00:00-04:00,54.234001,54.983334,53.445999,53.720669,108834000,0.0,0.0,56.399276,38.818730,2,0,NaN,1
268,2020-05-29 00:00:00-04:00,53.916668,55.666668,53.613998,55.666668,176467500,0.0,0.0,62.556463,39.041881,2,0,NaN,1
269,2020-06-01 00:00:00-04:00,57.200001,59.933334,56.939999,59.873333,224092500,0.0,0.0,71.910782,39.317794,2,0,NaN,1
270,2020-06-02 00:00:00-04:00,59.646667,60.577332,58.066666,58.770668,203484000,0.0,0.0,67.116137,39.575448,2,0,NaN,1
271,2020-06-03 00:00:00-04:00,59.208000,59.862667,58.673332,58.863998,119242500,0.0,0.0,67.317346,39.830926,2,0,NaN,1
272,2020-06-04 00:00:00-04:00,59.325333,59.716667,57.229332,57.625332,133315500,0.0,0.0,61.839025,40.066613,2,0,NaN,1
